In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

In [ ]:
from api_keys import g_key

In [3]:
# Read in data from weatherpy
cities_df = pd.read_csv("Data/cities.csv")
cities_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hemisphere,Country,Date
0,bredasdorp,-34.5322,20.0403,60.80,88.0,14.0,4.61,Southern Hemisphere,ZA,1.613530e+09
1,busselton,-33.6500,115.3333,75.99,65.0,0.0,3.00,Southern Hemisphere,AU,1.613530e+09
2,deputatskiy,69.3000,139.9000,-19.28,87.0,93.0,0.76,Northern Hemisphere,RU,1.613530e+09
3,potiskum,11.7091,11.0694,62.92,26.0,0.0,9.98,Northern Hemisphere,NG,1.613530e+09
4,port alfred,-33.5906,26.8910,62.87,79.0,3.0,14.25,Southern Hemisphere,ZA,1.613530e+09
...,...,...,...,...,...,...,...,...,...,...
582,wadi maliz,36.4681,8.5495,44.69,60.0,0.0,4.18,Northern Hemisphere,TN,1.613530e+09
583,arkhangelsk,64.5401,40.5433,-25.60,68.0,0.0,2.42,Northern Hemisphere,RU,1.613530e+09
584,el sauce,12.8869,-86.5390,76.10,72.0,66.0,0.25,Northern Hemisphere,NI,1.613530e+09
585,kiunga,-6.1219,141.2906,86.86,58.0,94.0,2.42,Southern Hemisphere,PG,1.613530e+09


In [4]:
# Configure gmaps with API key
gmaps.configure(g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Latitude", "Longitude"]].astype(float)
humidity = cities_df["Humidity (%)"].astype(float)

In [11]:
# Create a humidity Heatmap layer

# Use coordinates to center the map and zoom
coordinates = (0, 0)

# Use max humidity for max_intensity parameter in heatmap_layer
max_humidity = cities_df["Humidity (%)"].max()

# Build heat map
fig = gmaps.figure(center=coordinates, zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=max_humidity,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Find only cities with ideal vacation conditions
hotel_df = cities_df.loc[((cities_df['Temperature (F)'] < 75)&(cities_df['Wind Speed (mph)'] < 10)&(cities_df['Temperature (F)'] >= 70)&(cities_df['Cloudiness (%)'] == 0))]

100.0

In [8]:
# parameters for api call
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#column in df for storing hotel names
hotel_df['name'] = ""

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get restaurant type from df
    lat = row['Latitude']
    long = row['Longitude']
    location = f"{lat}, {long}"

    # add keyword to params dict
    params['location'] = location
    
    # assemble url and make API request
    hotel_data = requests.get(base_url, params=params).json()
    hotel_info = hotel_data['results']
    
    # try to store data
    try:
        hotel_df.loc[index, 'name'] = hotel_info[0]['name']
    except(KeyError, IndexError):
        print(f"Unable to find hotel name for {row['City']}")

C:\Users\nabul\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\nabul\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Unable to find hotel name for pochutla


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Create hotel marker layer
hotel_layer = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))